# Cái này là để cài đặt folder thành pakege để gọi imoport mà ko lỗi á. Nó tự sinh ra file 'signature_verification.egg-info' chứa thông tin tác giả của mìnhmình

In [2]:
%cd ..

# Cài đặt package
!pip install -e .

# Import module
from utils.helpers import load_config
from losses.contrastive_loss import ContrastiveLoss
from models.siamese_network import SiameseNetwork

print("Package installed and imported successfully!")

C:\Users\HP\PycharmProjects


ERROR: file:///C:/Users/HP/PycharmProjects does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Obtaining file:///C:/Users/HP/PycharmProjects
Package installed and imported successfully!


In [3]:
import os
import yaml
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
import random
from torch.utils.data import Dataset
from PIL import Image

class SiameseSignatureDataset(Dataset):
    def __init__(self, org_dir, forg_dir, transform=None):
        self.org_images = sorted([os.path.join(org_dir, f) for f in os.listdir(org_dir) if f.endswith('.png')])
        self.forg_images = sorted([os.path.join(forg_dir, f) for f in os.listdir(forg_dir) if f.endswith('.png')])
        self.transform = transform
        self.pairs = self._create_pairs()

    def _create_pairs(self):
        pairs = []
        # Tạo positive pairs (cùng người, chữ ký thật với thật)
        for i in range(len(self.org_images)):
            # Lấy 2 chữ ký khác nhau của cùng 1 người (dựa vào tên file)
            base_name = os.path.basename(self.org_images[i]).split('_')[1]
            matching = [img for img in self.org_images if f"_{base_name}_" in img]
            if len(matching) > 1:
                pairs.append((random.choice(matching), self.org_images[i], 1))  # Label 1 = similar

        # Tạo negative pairs (chữ ký thật với giả hoặc khác người)
        for i in range(len(self.org_images)):
            # Chữ ký thật với giả của cùng người
            base_name = os.path.basename(self.org_images[i]).split('_')[1]
            forg_match = [img for img in self.forg_images if f"_{base_name}_" in img]
            if forg_match:
                pairs.append((self.org_images[i], random.choice(forg_match), 0))  # Label 0 = dissimilar

            # Chữ ký của 2 người khác nhau
            if i < len(self.org_images) - 1:
                pairs.append((self.org_images[i], self.org_images[i+1], 0))

        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]

        img1 = Image.open(img1_path).convert('L')
        img2 = Image.open(img2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return (img1, img2), torch.tensor(label, dtype=torch.float32)

In [5]:
from torch.utils.data import DataLoader
# Transform chung
transform = transforms.Compose([
    transforms.Resize((220, 150)),
    transforms.Grayscale(),  # Đảm bảo ảnh 1 kênh
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 1 kênh -> 3 kênh
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Tạo dataset
siamese_dataset = SiameseSignatureDataset(
    org_dir=r'C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\signatures\full_org',
    forg_dir=r'C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\signatures\full_forg',
    transform=transform
)

# Chia train/test
train_size = int(0.8 * len(siamese_dataset))
siamese_train, siamese_test = torch.utils.data.random_split(
    siamese_dataset, [train_size, len(siamese_dataset) - train_size]
)

# Tạo DataLoader
siamese_train_loader = DataLoader(siamese_train, batch_size=32, shuffle=True)
siamese_test_loader = DataLoader(siamese_test, batch_size=32, shuffle=False)


In [6]:
# Kiểm tra Siamese Dataset
print(f"Siamese Dataset - Total pairs: {len(siamese_dataset)}")
sample_pair, label = siamese_dataset[0]
print(f"Pair shape: {sample_pair[0].shape}, {sample_pair[1].shape}")
print(f"Label: {label}")

Siamese Dataset - Total pairs: 3959
Pair shape: torch.Size([3, 220, 150]), torch.Size([3, 220, 150])
Label: 1.0


In [7]:
#Nayf ae xem sửa thửthử
config = load_config(r'C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\configs\config_siamese.yaml')
print(config)

{'model': {'backbone': 'resnet18', 'feature_dim': 512}, 'training': {'batch_size': 32, 'learning_rate': 0.001, 'optimizer': 'adam', 'num_epochs': 50, 'margin': 0.5, 'scheduler': 'cosine'}, 'dataset': {'train_data_path': './data/triplet/train/', 'val_data_path': './data/triplet/val/', 'input_size': '(220,150)'}, 'device': 'cuda', 'logging': {'log_dir': './logs/', 'checkpoint_dir': './checkpoints/', 'save_freq': 5}}


In [8]:
#Tiếp theo định nghĩa model từ con config
model = SiameseNetwork(config['model']['backbone'], config['model']['feature_dim'])

optimizer = optim.Adam(model.parameters(), lr=config['training']['learning_rate'])
device = torch.device(config['device'] if torch.cuda.is_available() else "cpu")
model.to(device)

C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SiameseNetwork(
  (feature_extractor): ResNetFeatureExtractor(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

AE xem thử rồi cấu trúc ổn không.

In [12]:
from losses import contrastive_loss
#Train kiểu kiểu như vầy ae cứ xem tham thảo

num_epochs = config['training']['num_epochs']
checkpoint_dir = config['logging']['checkpoint_dir']
save_freq = config['logging']['save_freq']

# Tạo thư mục lưu checkpoint nếu chưa có
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for (img1, img2), label in siamese_train_loader:
        img1, img2 = img1.to(device), img2.to(device)
        label = label.to(device)

        # Forward pass
        output1,output2 = model(img1,img2)

        # Tính loss
        loss = ContrastiveLoss()(output1, output2, label)

        # Backward và optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(siamese_train_loader)
    print(f"Epoch [{epoch}/{num_epochs}] - Loss: {avg_loss:.4f}")

    # === SAVE CHECKPOINT ===
    if epoch % save_freq == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")


KeyboardInterrupt: 